In [ ]:
from llama_index.core.agent.workflow import ReActAgent, FunctionAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.workflow import Context
import requests
import os
from typing import List, Union
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Constants
DATA_API = os.getenv("DATA_API")

# Shared function to fetch and process holder names
def fetch_holder_names() -> Union[List[str], str]:
    """Fetches 'holder_name' facets from dati.gov.it API."""
    response = requests.get(DATA_API)
    if response.status_code == 200:
        data = response.json()
        if data.get("success"):
            return data.get("result", {}).get("facets", {}).get("holder_name", {}).keys()
        return "Errore: richiesta non riuscita."
    return f"Errore HTTP: {response.status_code}"

# Filtering logic
def filter_holder_names(data: List[str], search: str) -> List[str]:
    """Returns holder names containing the search string."""
    return [name for name in data if search.lower() in name.lower()]

# Tool: get all holder names
def dataset_tool() -> List[str]:
    """Tool: Retrieve all holder names."""
    result = fetch_holder_names()
    return list(result) if isinstance(result, dict) else result

# Tool: filter holder names
def holder_name_tool(data: List[str], holder_name: str) -> List[str]:
    """Tool: Filter holder names by keyword."""
    return filter_holder_names(data, holder_name)

# Tool: count holder names by keyword
def sum_holder_names(data: List[str], holder_name: str) -> int:
    """Tool: Count holder names matching a keyword."""
    return len(filter_holder_names(data, holder_name))


## react agent

In [ ]:
# Setup LLM and Agent
llm = OpenAI(model="gpt-3.5-turbo")
agent = ReActAgent(
    tools=[dataset_tool, holder_name_tool, sum_holder_names],
    llm=llm,
    verbose=True
)
ctx = Context(agent)

# Run Agent
handler = agent.run("Dimmi il nome di alcune università? Quali altri enti ci sono?", ctx=ctx)

## function agent

In [ ]:
# Setup LLM and Agent
llm = OpenAI(model="gpt-3.5-turbo")
agent = FunctionAgent(
    tools=[dataset_tool, holder_name_tool, sum_holder_names],
    llm=llm,
    verbose=True,
    name="DatiGovAgent",
    description="Agent to interact with dati.gov.it API."
)
ctx = Context(agent)

# Run Agent
handler = agent.run("Dimmi il nome di alcune università? Quali altri enti ci sono?", ctx=ctx)

In [ ]:
from llama_index.core.agent.workflow import AgentStream

async for ev in handler.stream_events():
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler